In [9]:
#!pip install transformers

In [12]:
from transformers import BertConfig, BertTokenizer, BertModel
from torch import nn
from tqdm import tqdm

import torch

In [13]:
bert_name = 'bert-base-uncased'

In [15]:
config = BertConfig.from_pretrained(bert_name)
tokenizer = BertTokenizer.from_pretrained(bert_name)
model = BertModel.from_pretrained(bert_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
config

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

In [17]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [18]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [52]:
sentence = "I want to go home."
output = tokenizer(sentence)

In [20]:
output

{'input_ids': [101, 1045, 2215, 2000, 2175, 2188, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
tokenized = tokenizer.tokenize(sentence)

In [22]:
tokenized

['i', 'want', 'to', 'go', 'home', '.']

In [23]:
vocab = tokenizer.get_vocab()

In [25]:
#print(vocab)
print(len(vocab))

30522


In [26]:
print(vocab['[CLS]'])
print(vocab['[SEP]'])

101
102


In [28]:
token_ids = [vocab[token] for token in tokenized]
print(token_ids)

[1045, 2215, 2000, 2175, 2188, 1012]


In [30]:
token_ids = [tokenizer._convert_token_to_id(token) for token in tokenized]
print(token_ids)

[1045, 2215, 2000, 2175, 2188, 1012]


In [32]:
token_ids = tokenizer.convert_tokens_to_ids(tokenized)
print(token_ids)

[1045, 2215, 2000, 2175, 2188, 1012]


In [53]:
token_ids = tokenizer.encode(sentence)
print(token_ids)
a = tokenizer.convert_ids_to_tokens(token_ids)
print(a)

[101, 1045, 2215, 2000, 2175, 2188, 1012, 102]
['[CLS]', 'i', 'want', 'to', 'go', 'home', '.', '[SEP]']


In [51]:
sentence = tokenizer.convert_tokens_to_string(tokenized)
print(sentence)

i want to go home .


In [37]:
tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(tokens)
sentence = tokenizer.convert_tokens_to_string(tokens)
print(sentence)

['[CLS]', 'i', 'want', 'to', 'go', 'home', '.', '[SEP]']
[CLS] i want to go home . [SEP]


In [45]:
out = tokenizer("I want to go home.", "Me too.")
a = tokenizer.convert_ids_to_tokens(out['input_ids'])
print(a)

['[CLS]', 'i', 'want', 'to', 'go', 'home', '.', '[SEP]', 'me', 'too', '.', '[SEP]']


In [46]:
data = [
  "I want to go home.",
  "My dog's name is Max.",
  "Natural Language Processing is my favorite research field.",
  "Welcome. How can I help you?",
  "Shoot for the moon. Even if you miss, you'll land among the stars."
]

In [47]:
max_len = 0
batch = []

for sent in tqdm(data):
    token_ids = tokenizer.encode(sent)
    max_len = max(max_len, len(token_ids))
    batch.append(token_ids)

100%|██████████| 5/5 [00:00<00:00, 2400.04it/s]


In [54]:
pad_id = tokenizer._convert_token_to_id('[PAD]')

for i, token_ids in enumerate(tqdm(batch)):
    if len(token_ids) < max_len:
        batch[i] = token_ids + [pad_id] * (max_len - len(token_ids))

100%|██████████| 5/5 [00:00<00:00, 46091.25it/s]


In [55]:
batch = torch.LongTensor(batch)

print(batch)
print(batch.shape)

tensor([[ 101, 1045, 2215, 2000, 2175, 2188, 1012,  102,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 2026, 3899, 1005, 1055, 2171, 2003, 4098, 1012,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 3019, 2653, 6364, 2003, 2026, 5440, 2470, 2492, 1012,  102,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 6160, 1012, 2129, 2064, 1045, 2393, 2017, 1029,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0],
        [ 101, 5607, 2005, 1996, 4231, 1012, 2130, 2065, 2017, 3335, 1010, 2017,
         1005, 2222, 2455, 2426, 1996, 3340, 1012,  102]])
torch.Size([5, 20])


In [57]:
batch_mask = (batch != pad_id).float()

print(batch_mask)
print(batch.shape)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1.]])
torch.Size([5, 20])


In [58]:
outputs = model(input_ids=batch, attention_mask=batch_mask)

In [66]:
outputs

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0350,  0.3950, -0.0622,  ..., -0.0456,  0.2563,  0.5969],
         [ 0.2861,  0.5091, -0.1923,  ...,  0.0136,  0.4276,  0.4311],
         [ 0.1017,  0.3032,  1.1099,  ..., -0.0641, -0.0841,  0.3642],
         ...,
         [-0.0647,  0.2084,  0.2231,  ...,  0.3165, -0.1867,  0.1380],
         [ 0.1437,  0.3288,  0.3981,  ...,  0.0445, -0.2554,  0.2308],
         [ 0.2338,  0.2403,  0.3440,  ...,  0.0508, -0.2114,  0.0998]],

        [[-0.1119,  0.2266, -0.2985,  ..., -0.2968,  0.5495,  0.5525],
         [-0.0327,  0.1727, -0.3103,  ..., -0.1726,  0.7786,  0.2142],
         [ 0.6370,  0.3274,  0.1777,  ..., -1.0487,  0.7025,  0.0337],
         ...,
         [-0.2139, -0.0164,  0.1756,  ...,  0.1646, -0.0521, -0.0087],
         [-0.2137, -0.0305,  0.1925,  ...,  0.1855,  0.0185, -0.0185],
         [-0.4911, -0.2284, -0.0021,  ...,  0.5877,  0.5304, -0.3678]],

        [[-0.0133,  0.0811, -0.5912,  ..., -0.1440,  

In [60]:
# B: batch size, L: max length, d_h: hidden size
last_hidden_states = outputs[0]  # (B, L, d_h)

print(last_hidden_states.shape)

torch.Size([5, 20, 768])


In [63]:
pooler_output = outputs[1]

print(pooler_output.shape)

torch.Size([5, 768])


In [67]:
num_classes = 10

sent_linear = nn.Linear(config.hidden_size, num_classes)

In [68]:
cls_output = last_hidden_states[:,0,:]

print(cls_output)
print(cls_output.shape)

tensor([[ 0.0350,  0.3950, -0.0622,  ..., -0.0456,  0.2563,  0.5969],
        [-0.1119,  0.2266, -0.2985,  ..., -0.2968,  0.5495,  0.5525],
        [-0.0133,  0.0811, -0.5912,  ..., -0.1440,  0.1487,  0.6923],
        [-0.3544, -0.0152, -0.1947,  ..., -0.3146,  0.1046,  0.5122],
        [-0.0181,  0.0981,  0.0771,  ..., -0.2977, -0.1505,  0.6382]],
       grad_fn=<SliceBackward>)
torch.Size([5, 768])


In [69]:
sent_output = sent_linear(cls_output)

print(sent_output)
print(sent_output.shape)

tensor([[ 2.2361e-01, -1.3889e-01,  8.4721e-01, -1.8188e-01,  1.4421e-01,
          1.7186e-01, -2.6758e-01,  3.3330e-01, -6.9772e-02, -3.1956e-01],
        [-6.1222e-02, -3.5865e-02,  7.2794e-01, -1.4166e-01,  6.1714e-02,
          1.2698e-01, -1.1617e-01,  3.8381e-01, -1.0419e-01, -5.4432e-01],
        [-1.2115e-01, -4.5172e-02,  9.0202e-01, -9.5614e-02,  7.0386e-02,
          2.9725e-01, -2.3940e-01,  1.6299e-01, -7.0256e-04, -3.5576e-01],
        [-4.8322e-02,  6.3114e-02,  9.3144e-01, -2.2633e-01,  1.2165e-01,
          4.2386e-01, -5.7682e-02,  4.1699e-01, -4.7099e-01, -2.4045e-01],
        [ 9.3797e-02, -1.7645e-01,  3.7345e-01, -1.6578e-01,  8.3976e-02,
          1.7826e-01, -3.2605e-01,  2.2463e-01, -1.2700e-01, -3.5611e-01]],
       grad_fn=<AddmmBackward>)
torch.Size([5, 10])


In [70]:
num_classes = 50

token_linear = nn.Linear(config.hidden_size, num_classes)

In [71]:
token_output = token_linear(last_hidden_states)

print(token_output)
print(token_output.shape)

tensor([[[ 1.0681e-01, -6.4778e-01, -2.7655e-02,  ..., -3.2080e-01,
          -1.6857e-01, -3.0121e-01],
         [-1.0705e-01, -2.8840e-01, -1.7258e-01,  ..., -1.3158e-01,
          -5.8157e-02, -7.4053e-02],
         [ 3.8048e-02,  6.7307e-02, -4.1661e-01,  ...,  9.7741e-02,
          -2.3464e-01,  2.0872e-01],
         ...,
         [-1.2714e-01, -2.4972e-01, -2.1255e-01,  ..., -1.3465e-01,
          -5.3657e-04,  9.9420e-03],
         [-1.7029e-01, -1.9518e-01, -2.1328e-01,  ..., -1.9462e-01,
          -9.3710e-02,  5.8967e-02],
         [-2.3694e-01, -1.8848e-01, -2.4006e-01,  ..., -2.5876e-01,
          -9.8043e-02,  7.6935e-02]],

        [[ 9.8870e-02, -3.9936e-01, -1.2400e-01,  ..., -6.0300e-01,
           1.1112e-01, -4.2589e-01],
         [ 1.4138e-01,  3.0952e-01, -2.6215e-01,  ..., -4.3790e-01,
           1.9369e-01, -3.7250e-02],
         [ 2.7876e-01, -4.8707e-02, -5.5234e-01,  ..., -2.7661e-01,
          -5.7615e-02,  8.0049e-02],
         ...,
         [ 7.2401e-02, -7

In [72]:
seq_model = BertForSequenceClassification.from_pretrained(bert_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [73]:
seq_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [74]:
lm_model = BertForMaskedLM.from_pretrained(bert_name, config=config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
lm_model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr